In [1]:
! which python

/Users/no_lineal/opt/anaconda3/envs/music38/bin/python


In [2]:
import torch
from torch import nn
import torch.nn.functional as F

import os

In [3]:
"""

    where am i?

"""

PATH = os.getcwd() + '/'
data_path = PATH + 'data/nesmdb_midi/'

train_path = data_path + 'train/'

output_path =  PATH + 'output/'

print(f'path: {PATH}')
print(f'data path: {data_path}')
print(f'train path: {train_path}')
print(f'output path: {output_path}')

path: /Users/no_lineal/Documents/GitHub/mir_project/
data path: /Users/no_lineal/Documents/GitHub/mir_project/data/nesmdb_midi/
train path: /Users/no_lineal/Documents/GitHub/mir_project/data/nesmdb_midi/train/
output path: /Users/no_lineal/Documents/GitHub/mir_project/output/


In [4]:
"""

    device

"""

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f'device: {device}')

device: cpu


In [5]:
"""

    encode/decode

"""

class Encoder(nn.Module):
    
    def __init__(self, latent_dims):
        
        super(Encoder, self).__init__()
        
        self.linear1 = nn.Linear(784, 512)
        self.linear2 = nn.Linear(512, latent_dims)

    def forward(self, x):
        
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.linear1(x))
        
        return self.linear2(x)

class Decoder(nn.Module):
    
    def __init__(self, latent_dims):
        
        super(Decoder, self).__init__()
        
        self.linear1 = nn.Linear(latent_dims, 512)
        self.linear2 = nn.Linear(512, 784)

    def forward(self, z):
        
        z = F.relu(self.linear1(z))
        z = torch.sigmoid(self.linear2(z))
        
        return z.reshape((-1, 1, 28, 28))

In [6]:
"""

    autoencoder

"""

class Autoencoder(nn.Module):
    
    def __init__(self, latent_dims):
        
        super(Autoencoder, self).__init__()
        
        self.encoder = Encoder(latent_dims)
        self.decoder = Decoder(latent_dims)

    def forward(self, x):
        
        z = self.encoder(x)
        
        return self.decoder(z)

In [7]:
"""

    train autoencoder

"""

def train(autoencoder, data, epochs=20):
    
    opt = torch.optim.Adam(autoencoder.parameters())
    
    for epoch in range(epochs):
        
        for x, y in data:
            
            x = x.to(device) # GPU
            opt.zero_grad()
            x_hat = autoencoder(x)
            loss = ((x - x_hat)**2).sum()
            loss.backward()
            opt.step()
            
    return autoencoder